In [1]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

In [2]:
#installing dependencies
!pip install pandas==1.4.4
!pip install numpy==1.24.4
!pip install apache_beam==2.56.0
!pip install opensearch==2.1.0
!pip install langchain==0.1.14 #used for chunking

ERROR: Could not find a version that satisfies the requirement opensearch==2.1.0 (from versions: 0.5, 0.6, 0.7, 0.8, 0.9, 0.9.1, 0.9.2)
ERROR: No matching distribution found for opensearch==2.1.0


In [1]:
import apache_beam as beam
import pandas as pd
from apache_beam.ml.transforms.base import MLTransform
from apache_beam.transforms.enrichment import Enrichment
from apache_beam.ml.transforms.embeddings.huggingface import SentenceTransformerEmbeddings
import tempfile
import json

from opensearchpy import OpenSearch
from opensearch_connector import *
from opensearch_enrichment import *
from chunks_generation import *

In [4]:
#To check beam version installed 
beam.__version__

'2.56.0'

Next we load the Opensearch credentials that will be needed in access during following steps. It reads credentials from environment. Please replace the values in the file before running this.

In [2]:
from dotenv import load_dotenv

load_dotenv('credentials.env')

True

# Reading json data

In [3]:
with open('hf_small_wikipedia.json', 'r') as j:
     contents = json.loads(j.read())

In [7]:
type(contents)

list

# Creating a Search Index¶
### Next few cells will show how to specify and create a search index in OpenSearch vector DB. 

Set some constants for defining our index like the distance metric and the index name.

Define the index schema with OpenSearchSearch fields.

Create the index.

Index creation is neeeded only once.

In [13]:
http_auth = [os.environ['OPENSEARCH_USERNAME'], os.environ['OPENSEARCH_PASSWORD']]
client = OpenSearch(hosts = ['https://localhost:9200'],
                    http_auth = http_auth,
                    verify_certs = False
                    )

index_name = 'embeddings-index'

index_body = {
  "settings": {
      "index.knn": True
   },
   "mappings": {
      "properties": {
         "title_vector": {
            "type": "knn_vector",
            "dimension": 384,
            "method": {
               "engine": "faiss",
               "name": "hnsw"
            }
         },
         "text_vector": {
            "type": "knn_vector",
            "dimension": 384,
            "method": {
               "engine": "faiss",
               "name": "hnsw"
            }
         },
         "id": {
            "type": "long"
         },
         "url": {
            "type": "text"
         },
         "title": {
            "type": "text"
         },
         "text": {
            "type": "text"
         },
         "section_id": {
            "type": "long"
         }
    }
}}

response = client.indices.create(index_name, body=index_body)

# Command for deleting an index
# response = client.indices.delete(index = index_name)

/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:opensearch:PUT https://localhost:9200/embeddings-index [status:200 request:0.351s]


# Creating Knowledge Base in OpenSearch Vector Database¶
After creating a search index, we can load documents into it. We will use the same documents we used in the previous cell. 

In [5]:
#Insertion Pipeline
import os

artifact_location = tempfile.mkdtemp()
generate_embedding_fn = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2',
                                                               columns=['title','text'])
with beam.Pipeline() as p:
    embeddings = (
        p  
        | "Read data" >> beam.Create(contents)
        | "Generate text chunks" >> ChunksGeneration(chunk_size = 500, chunk_overlap = 0, chunking_strategy = ChunkingStrategy.SPLIT_BY_TOKENS)
        | "Insert document in openSearch" >> InsertDocInOpenSearch(host='https://localhost', username=os.environ['OPENSEARCH_USERNAME'], password=os.environ['OPENSEARCH_PASSWORD'], port=9200, batch_size = 10)
        | "Generate Embeddings" >> MLTransform(write_artifact_location=artifact_location).with_transform(generate_embedding_fn) 
        | "Insert Embedding in openSearch" >> InsertEmbeddingInOpenSearch(host='https://localhost', username=os.environ['OPENSEARCH_USERNAME'], password=os.environ['OPENSEARCH_PASSWORD'], port=9200, batch_size = 10, embedded_columns=['title','text'])
    )


INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.


INFO:apache_beam.runners.worker.statecache:Creating state cache with size 104857600
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
2024-08-17 12:28:49.961311: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTr

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:opensearch_connector:Adding Docs in DB: 9
/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:opensearch:POST https://localhost:9200/_bulk?refresh=true [status:200 request:0.066s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:BatchElements statistics: element_count=349 batch_count=27 next_batch_size=4 timings=[(1, 0.18766427040100098), (2, 0.5434770584106445), (1, 0.18743109703063965), (2, 0.28671717643737793), (1, 0.180436372756958), (2, 0.2946739196777344), (1, 0.23954296112060547), (2, 0.2287900447845459), (4, 0.6388511657714844), (6, 0.7156858444213867), (7, 0.8332958221435547), (13, 1.1533119678497314), (12, 1.2309019565582275), (14, 1.1411330699920654), (19, 1.4424619674682617), (19, 1.1780219078063965), (24, 1.7884211540222168), (19, 1.191762924194336), (26, 1.8830091953277588), (32, 2.3361740112304688), (29, 1.9658150672912598), (26, 1.5100517272949219), (26, 1.811798095703125), (32, 2.051542043685913), (26, 1.5136959552764893), (2, 0.44672513008117676)]
INFO:opensearch_connector:Insert Embeddings in opensearch DB, count=10
/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certi

### Pipeline Steps:
Now that we have ingested the documents in OpenSearch, we will create a embeddings transform, which is used for storing the text and its embedding in openSearch vector db

# Running Search Queries/ Perform Enrichment

### Pipeline Steps:
Create a search transform, which emits the document Id, vector score along with the matching text from knowledge base

In [6]:
# Enchriment Pipeline

data = [{'text':'What is climate ?'}]

artifact_location = tempfile.mkdtemp()
generate_embedding_fn = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2',
                                                                columns=['text'])

opensearch_handler = OpenSearchEnrichmentHandler(opensearch_host='https://localhost', 
                                                 opensearch_port=9200, 
                                                 username = "admin", 
                                                 password = "admin")
                                       

with beam.Pipeline() as p:
  _ = (
      p
      | "Create" >> beam.Create(data)
      | "Generate Embedding" >> MLTransform(write_artifact_location=artifact_location).with_transform(generate_embedding_fn)
      | "Enrich W/ OpenSearch" >> Enrichment(opensearch_handler)
      | "Print" >> beam.Map(print)
  )

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:apache_beam.runners.worker.statecache:Creating state cache with size 104857600
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
INFO:opensearch:POST https://localhost:9200/embeddings-index/_search [status:200 request:0.315s]
INFO:opensearch_enrichment:Enrichment_results
INFO:root:BatchElements statistics: element_count=1 batch_count=1 next_batch_size=1 timings=[]


Row(text=[0.005681279115378857, 0.09814242273569107, 0.012122008018195629, 0.0887078046798706, -0.032298486679792404, 0.010596378706395626, 0.01474417932331562, -0.04602375999093056, -0.012696374207735062, 0.028775634244084358, -0.012717711739242077, -0.16198472678661346, -0.003693335922434926, -0.015376407653093338, -0.004146557301282883, -0.07635108381509781, -0.06286586076021194, -0.036414001137018204, -0.021039914339780807, -0.062046296894550323, -0.041434209793806076, 0.08710915595293045, -0.046050168573856354, 0.056758005172014236, -0.009660243056714535, -0.0029580420814454556, -0.04717544838786125, 0.09012085944414139, -0.05081005021929741, 0.09703554213047028, -0.011215518228709698, 0.12284161895513535, 0.030584124848246574, 0.007557293400168419, -0.06626550853252411, 0.0706571638584137, 0.021545734256505966, -0.0142928846180439, -0.06792338937520981, 0.033220842480659485, -0.0486346036195755, -0.04314371198415756, 0.031225889921188354, -0.014775502495467663, 0.0188262276351451

# Conclusion
Here we have demonstrated how we can implement Ingestion and Enrichment pipeline using OpenSearch vector DB by using ML Transfrom's SentenceTransformerEmbeddings for generating the embeddings of the text chunks.